### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import os
import pravega.grpc_gateway as pravega
from itertools import islice

In [ ]:
import pravega.video as video
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer
imp.reload(video);

### Define Pravega stream parameters

In [ ]:
#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
#gateway = '10.243.37.108:80'
scope = 'examples'
input_stream_name = 'multi-video-grid-input'
output_stream_name = 'multi-video-grid-output'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### Build timestamp index

In [ ]:
output_stream = IndexedStream(pravega_client, scope, output_stream_name)

#### Batch index builder (stop at tail)

In [ ]:
%%time
output_stream.update_index(force_full=True)

#### Show number of records in the index, first and last index records

In [ ]:
len(output_stream.index_df)

In [ ]:
output_stream.index_df.iloc[[0,-1]].T

### Video Player

In [ ]:
player = VideoPlayer(pravega_client=pravega_client, scope=scope, stream=output_stream)
player.interact()